In [2]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["about the author", "acknowledgements", "about the book", "table of content"]

/Users/andreafavia/miniconda3/envs/bookai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_classification(text, eq=None, threshold=0.3):
    assert eq in candidate_labels if eq else True
    if not eq:
        return classifier(text, candidate_labels=candidate_labels)["labels"][0]
    else:
        res = classifier(text, candidate_labels=candidate_labels)
        return res["labels"][0] == eq if res["scores"][0] > threshold else None

In [4]:
classifier("This is a book about the author", candidate_labels=candidate_labels)

{'sequence': 'This is a book about the author',
 'labels': ['about the author',
  'about the book',
  'table of content',
  'acknowledgements'],
 'scores': [0.5708119869232178,
  0.41445258259773254,
  0.007752928417176008,
  0.006982521153986454]}

In [5]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
from collections import defaultdict


# Function to read EPUB file
def read_epub(file_path, min_len_chapter=1000):
    chapter_cnt = 0
    result = defaultdict()
    book = epub.read_epub(file_path)

    result["title_of_book"] = book.title

    for item in book.get_items():
        # Check if item is of type document
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_body_content(), "html.parser")
            text_content = soup.get_text()

            if (
                not result.get("about_the_author")
                and len(text_content) < min_len_chapter
                and get_classification(text_content, "about the author")
            ):
                result["about_the_author"] = text_content
            elif (
                not result.get("acknowledgements")
                and len(text_content) < min_len_chapter
                and get_classification(text_content, "acknowledgements")
            ):
                result["acknowledgements"] = text_content
            elif (
                not result.get("about_the_book")
                and len(text_content) < min_len_chapter
                and get_classification(text_content, "about the book")
            ):
                result["about_the_book"] = text_content
            elif (
                not result.get("table_of_content")
                and len(text_content) < min_len_chapter
                and get_classification(text_content, "table of content")
            ):
                result["table_of_content"] = text_content
            elif len(text_content) > min_len_chapter:
                result[f"chapter_{chapter_cnt}"] = text_content
                chapter_cnt += 1

    return result, item


# Path to your EPUB file
file_path = "/Users/andreafavia/development/bookai/files/The Responsible Company.epub"


# Read and display the content
epub_content, item = read_epub(file_path)
print(epub_content)

/Users/andreafavia/miniconda3/envs/bookai/lib/python3.10/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/andreafavia/miniconda3/envs/bookai/lib/python3.10/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


defaultdict(None, {'title_of_book': 'The Responsible Company', 'acknowledgements': '\n\n\n', 'table_of_content': '\nALSO BY YVON CHOUINARD\nLet My People Go Surfing (2006)Climbing Ice (1982)\n', 'about_the_author': '\n“The most important right we have is the right to be responsible.”\n—Gerald Amos\n', 'chapter_0': '\nCHAPTER 1\nWhat We Do For a Living\nWE ARE ALL STILL in the earliest stages of learning how what we do for a living both threatens nature and fails to meet our deepest human needs. The impoverishment of our world and the devaluing of the priceless undermine our physical and economic well-being.\nYet the depth and breadth of technological innovation of the past few decades shows that we have not lost our most useful gifts: humans are ingenious, adaptive, clever. We also have moral capacity, compassion for life, and an appetite for justice. We now need to more fully engage these gifts to make economic life more socially just and environmentally responsible, and less destruct

In [ ]:
item.

'page-map'

In [ ]:
summary_path = "/Users/andreafavia/development/bookai/Big Feelings.json"

import json

with open(summary_path, "r") as f:
    data = json.load(f)
import pprint

for chapt, text in data.items():
    print("___")
    print(text)

___
Chapter 1 of the book explores the pervasive experience of uncertainty and the accompanying anxiety.  It begins with a personal narrative highlighting the agonizing nature of undiagnosed medical issues and the emotional toll of uncertainty.  This sets the stage for a discussion of the psychological factors contributing to anxiety: lack of control, unpredictability, and the perception of worsening circumstances.

The chapter then deconstructs common myths surrounding uncertainty.  It argues that certainty is unattainable, that anxiety doesn't always correlate with risk, and that resilience is not a panacea for all problems.  The authors emphasize the importance of acknowledging and accepting uncertainty as an inherent part of life, rather than striving for unrealistic control.

Practical strategies for navigating uncertainty are presented.  The authors advocate for "sitting with" anxiety, recognizing it as a protective mechanism, and reframing negative self-talk.  They encourage rea

In [ ]:
def generate_html_page(chapters):
    """
    Generates an HTML page with a stylish layout for the given chapters.

    Args:
      chapters: A dictionary where keys are chapter titles and values are chapter content.

    Returns:
      A string containing the HTML code for the page.
    """

    html = f"""
  <!DOCTYPE html>
  <html lang="en">
  <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>Book of Balance</title>
      <style>
          /* Basic Styling */
          body {{
              font-family: 'Open Sans', sans-serif;
              background-color: #f0f0f0;
              margin: 0;
              padding: 0;
              display: flex;
              flex-direction: column;
              min-height: 100vh;
          }}

          header {{
              background-color: #333;
              color: #fff;
              text-align: center;
              padding: 2rem 0;
          }}

          h1 {{
              font-size: 2.5rem;
              margin: 0;
          }}

          nav {{
              background-color: #eee;
              padding: 1rem 0;
          }}

          nav ul {{
              list-style: none;
              padding: 0;
              margin: 0;
              display: flex;
              justify-content: center;
          }}

          nav li {{
              margin: 0 1rem;
          }}

          nav a {{
              color: #333;
              text-decoration: none;
              font-weight: bold;
              transition: color 0.3s ease;
          }}

          nav a:hover {{
              color: #007bff;
          }}

          main {{
              flex-grow: 1;
              padding: 2rem;
              background-color: #fff;
              border-radius: 5px;
              box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
              margin: 1rem;
          }}

          h2 {{
              color: #333;
              margin-bottom: 1rem;
          }}

          footer {{
              background-color: #333;
              color: #fff;
              text-align: center;
              padding: 1rem 0;
              position: fixed;
              bottom: 0;
              width: 100%;
          }}
      </style>
  </head>
  <body>

      <header>
          <h1>Book of Balance</h1>
      </header>

      <nav>
          <ul>
  """

    for title in chapters:
        html += f"              <li><a href='#{title.replace(' ', '-')}'>{title}</a></li>\n"

    html += """
          </ul>
      </nav>

      <main>
  """

    for title, content in chapters.items():
        html += f"""
          <section id="{title.replace(' ', '-')}">
              <h2>{title}</h2>
              <p>{content}</p>
          </section>
  """

    html += """
      </main>

      <footer>
          &copy; 2023 Book of Balance
      </footer>

  </body>
  </html>
  """

    return html


# Example usage:
chapters = {
    "Chapter 1: Our Masturbation Machines": "This chapter explores the concept of addiction...",
    "Chapter 2: Running from Pain": "Chapter 2 explores the pervasive human tendency to avoid pain...",
    "Chapter 3: The Pleasure-Pain Balance": "Chapter 3 explores the interplay of pleasure and pain...",
}

html_content = generate_html_page(chapters)

# Save the HTML content to a file (optional)
with open("book_of_balance.html", "w") as f:
    f.write(html_content)

print("HTML page generated successfully!")

HTML page generated successfully!


In [19]:
def synthesize_text():
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    text = "Hello there."
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Neural2-D",
        ssml_gender=texttospeech.SsmlVoiceGender.MALE,
    )

    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

    response = client.synthesize_speech(request={"input": input_text, "voice": voice, "audio_config": audio_config})

    # The response's audio_content is binary.
    with open("output.mp3", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')

    return response

In [20]:
r = synthesize_text()

Audio content written to file "output.mp3"


In [ ]:
r.audio_content

b'\xff\xf3\x84\xc4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xf3\x84\xc4\x00%C\xae\x08\x00y\x86\xdd\x05A9\x02\x19\xc6\x87\xbex\xf2\xaa\xca\xfc\xee\x03\xc6\xb5_`\x839\x02\xe8\x00\x10\xd0\xf7\x0f\xb7I\x99\x8fnM=c1\x88\x10 C\x1c\xfa\x8f\xfcc\x93Oa\xd3\xf1\xafl\x83\xeeZ\x1d\xbf\x8c\xf8\xd1